## Converting ipyrad .u.str output for adegenet 

You need 3 files: the .u.str file from ipyrad, a file listing the population of each sample, and a file listing an integer for each population. Based on the format of my sample names (Population_ID), I created the Sample-Population file with a simple python script that takes in any of the stats output from Steps 2-6.You could also just manually make a file with a column of sample ids and a column of corresponding Population IDs.

Alternatively, if you already have a .geno file for all individuals, you can create a .ind file by listing all samples and their population information, then going straight to the splitGenoPops.py script.

Example of a .ind file:

Sample_A    U    Pop_1  
Sample_B    U    Pop_1  
Sample_C    U    Pop_2  
Sample_D    U    Pop_2  

In [ ]:
# %load /home/ksil91/Projects/Ostrea/makePopFile.py
##create population file for ipyrad. Can be used with ipyrad stats 
## files from steps 2-6.

import sys

def makePopFile(infile, outfile):
    IN = open(infile, "r")
    OUT = open(outfile, "w")
    for line in IN:
        sampleID = line.split()[0]
        popID = sampleID.split("_")[0]
        OUT.write(sampleID + " "+popID+"\n")
    IN.close()
    OUT.close()


def main(argv):
    #get arguments from command line
    inf = argv[1]
    outf = argv[2]
    makePopFile(inf,outf)

if __name__ == "__main__":
    status = main(sys.argv)
    sys.exit(status)



File should look like this, with sample id 1st, a space or tab, then the population id:

In [35]:
DIR = "/home/ksil91/Projects/Ostrea/"

In [54]:
!head /home/ksil91/Projects/Ostrea/over10k_popfile.txt

clusters_total clusters
BC1_1 BC1
BC1_10w_6 BC1
BC1_11 BC1
BC1_12 BC1
BC1_19 BC1
BC1_2 BC1
BC1_20 BC1
BC1_22 BC1
BC1_5 BC1


It's ok if you have some extra stuff on there for my subsequent code.

For .str files, you need to code the population as an integer. For this, I created a file with the Population ID string, the desired integer, and then optional columns of other info, like the full name of the population or GPS coordinates. This file can then be played around with to create different groupings of populations.

In [55]:
!head /home/ksil91/Projects/Ostrea/Pop2Int.txt

BC1 4 Victoria
BC2 1 Klaskino
BC3 2 Barkeley_Sound
BC4 3 Ladysmith
WA12 5 Discovery_Bay
WA11 6 Liberty_Bay
WA13 7 North_Bay
WA10 8 Triton_Cove
WA1 9 North_Willapa
WA9 9 South_Willpa


Then I made a script to add a column of integers to the .str ipyrad output corresponding to the population of each sample. This file can be used in the actual Structure program.

In [ ]:
# %load /home/ksil91/Projects/Ostrea/AddPopsStr.py
import sys

def addPops(str_infile, popfile, outfile, pop2int):
    IN = open(str_infile, "r")
    OUT = open(outfile, "w")
    pops = open(popfile, "r")
    pop2int = open(pop2int, "r")
    popdict = {}
    pop2intdict = {}
    for line in pop2int:
        popID = line.split()[0]
        intID = line.strip().split()[1]
        pop2intdict[popID] = intID
    for line in pops:
        sampleID = line.split()[0]
        popID = line.strip().split()[1]
        popdict[sampleID] = popID
    for line in IN:
        linelist = line.split()
        intID = pop2intdict[popdict[linelist[0]]]
        linelist.insert(1, intID)
        print >> OUT, "\t".join(str(e) for e in linelist)
    IN.close()
    OUT.close()
    pops.close()
    pop2int.close()

def main(argv):
    #get arguments from command line
    inf = argv[1]
    outf = argv[2]
    popf = argv[3]
    pop2int = argv[4]
    addPops(inf,popf, outf, pop2int)

if __name__ == "__main__":
    status = main(sys.argv)
    sys.exit(status)



In [14]:
%%sh
head /home/ksil91/Projects/Ostrea/over10k-BCWA-min75H25.u.str | cut -f 1-4

BC1_1	4	3	0
BC1_1	4	3	0
BC1_10w_6	4	3	0
BC1_10w_6	4	3	0
BC1_11	4	3	0
BC1_11	4	3	0
BC1_12	4	3	0
BC1_12	4	3	0
BC1_19	4	-9	0
BC1_19	4	-9	0


Population BC1 is given an integer identifation 

Adegenet will take strings as population identifiers, so I made a copy of the .str file with the 3rd column of my Pop2Int.txt file instead of the integers. To be able to load into adegenet, you also have to rename the .u.str file to be .stru.

In [102]:
def addPopNames(str_infile, popfile, outfile, pop2int):
    IN = open(str_infile, "r")
    OUT = open(outfile, "w")
    pops = open(popfile, "r")
    pop2int = open(pop2int, "r")
    popdict = {}
    pop2namedict = {}
    for line in pop2int:
        popID = line.split()[0]
        nameID = line.strip().split()[2]
        pop2namedict[popID] = nameID
    for line in pops:
        sampleID = line.split()[0]
        popID = line.strip().split()[1]
        popdict[sampleID] = popID
    for line in IN:
        linelist = line.split()
        intID = pop2namedict[popdict[linelist[0]]]
        linelist.insert(1, intID)
        print >> OUT, "\t".join(str(e) for e in linelist)
    IN.close()
    OUT.close()
    pops.close()
    pop2int.close()

addPopNames("/home/ksil91/Projects/Ostrea/over10k-min50H32.u.str","/home/ksil91/Projects/Ostrea/over10k_popfile.txt",\
            "/home/ksil91/Projects/Ostrea/over10k-min50H32popnames_u.stru", "/home/ksil91/Projects/Ostrea/Pop2Int.txt")

In [103]:
!head /home/ksil91/Projects/Ostrea/over10k-min50H32popnames_u.stru | cut -f 1-7


BC1_1	Victoria	3	-9	1	3	-9
BC1_1	Victoria	3	-9	1	3	-9
BC1_10w_6	Victoria	3	3	1	3	2
BC1_10w_6	Victoria	3	3	1	3	2
BC1_11	Victoria	3	-9	1	-9	-9
BC1_11	Victoria	3	-9	1	-9	-9
BC1_12	Victoria	3	3	-9	-9	2
BC1_12	Victoria	3	3	-9	-9	2
BC1_19	Victoria	3	-9	-9	-9	-9
BC1_19	Victoria	3	-9	-9	-9	-9


In [1]:
%cd ~/Projects/Ostrea/

/home/ksil91/Projects/Ostrea


In [11]:
%load_ext rpy2.ipython

After adding population info to the .str file, convert into Eigensoft format (.geno/.ind/.snp) files with PGDSpider2. Make sure you set the name for your .ind file and .snp file in PGDSpider settings! (*should figure out how to do on command line*) Afterwards, edit the .ind file to reflect the names you want for populations and PGDSpider gives them weird long names.

In [5]:
%%sh
##My edited .ind file
head over10k-min75H32u.ind

BC1_1_Victoria	U	Victoria
BC1_10w_6_Victoria	U	Victoria
BC1_11_Victoria	U	Victoria
BC1_12_Victoria	U	Victoria
BC1_19_Victoria	U	Victoria
BC1_2_Victoria	U	Victoria
BC1_20_Victoria	U	Victoria
BC1_22_Victoria	U	Victoria
BC1_5_Victoria	U	Victoria
BC1_7_Victoria	U	Victoria


## Split .geno file into pairs of populations.

In [ ]:
# %load splitGenoPops.py
##Creates a new .geno and .ind file for each pair combination of populations. This is to be used for Fst and Pi scripts.

import sys
from itertools import combinations


def getPopList(popfile):
    pop = open(popfile, "r")
    poplist = []
    samplelist = []
    for line in pop:
        sampleID, U, popID = line.strip().split()
        poplist.append(popID)
        samplelist.append(sampleID)
    pop.close()
    return(poplist,samplelist)

def makeNewGeno(suf,pop12,poplist,samplelist):
    IN = open(suf+".geno","r")
    OUT = open(suf+"_"+pop12[0]+"_"+pop12[1]+".geno","w")
    indfile = open(suf+"_"+pop12[0]+"_"+pop12[1]+".ind","w")
    for line in IN:
        genos = list(line.strip())
        for i in range(0,len(genos)):
            if poplist[i] in pop12:
                OUT.write(genos[i])
        OUT.write("\n")
    for i in range(0,len(samplelist)):
        if poplist[i] == pop12[0]:
            indfile.write(samplelist[i]+"    0\n")
        elif poplist[i] == pop12[1]:
            indfile.write(samplelist[i]+"    1\n")
    IN.close()
    OUT.close()
    indfile.close()


def main(argv):
    #get arguments from command line
    suf = argv[1]
    popfile = argv[2]
    poplist,samplelist = getPopList(popfile)
    popset = set(poplist)
    for c in combinations(popset,2):
        makeNewGeno(suf,c,poplist,samplelist)
    

if __name__ == "__main__":
    status = main(sys.argv)
    sys.exit(status)



In [31]:
%run splitGenoPops.py over10k-min50H32u over10k-min50H32u.ind

In [25]:
%ls over10k-min75H32u*

over10k-min75H32u_Barkeley_Sound_Conchaphila.geno
over10k-min75H32u_Barkeley_Sound_Conchaphila.ind
over10k-min75H32u_Barkeley_Sound_Discovery_Bay.geno
over10k-min75H32u_Barkeley_Sound_Discovery_Bay.ind
over10k-min75H32u_Barkeley_Sound_Elkhorn_SLough.geno
over10k-min75H32u_Barkeley_Sound_Elkhorn_SLough.ind
over10k-min75H32u_Barkeley_Sound_Humboldt.geno
over10k-min75H32u_Barkeley_Sound_Humboldt.ind
over10k-min75H32u_Barkeley_Sound_Klaskino.geno
over10k-min75H32u_Barkeley_Sound_Klaskino.ind
over10k-min75H32u_Barkeley_Sound_Ladysmith.geno
over10k-min75H32u_Barkeley_Sound_Ladysmith.ind
over10k-min75H32u_Barkeley_Sound_Liberty_Bay.geno
over10k-min75H32u_Barkeley_Sound_Liberty_Bay.ind
over10k-min75H32u_Barkeley_Sound_Mugu_Lagoon.geno
over10k-min75H32u_Barkeley_Sound_Mugu_Lagoon.ind
over10k-min75H32u_Barkeley_Sound_North_Bay.geno
over10k-min75H32u_Barkeley_Sound_North_Bay.ind
over10k-min75H32u_Barkeley_Sound_San_Diego.geno
over10k-min75H32u_Barkeley_Sound_San_Diego.ind
over10k-min75H32u_Barkel